In [1]:

# independent variables - 
# start from 1 min averages
# stop wherever it hangs

# concat other features
# concat averages
# concat 1 min rsis
# concat 1 min macd


# dependent features
# category

# write everything to s3 location
# train.py
# sagemaker notebook results

In [1]:
# convert this into pipeline

In [2]:
# experiment with settings
# correlation analysis

In [3]:
import yaml
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import pyarrow.dataset as ds
import pyarrow.parquet as pq
warnings.filterwarnings("ignore")

In [4]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/sisyphus/AthenaInsights/notebooks/data_cleaning'

In [5]:
def load_yaml(loc):
    with open(loc) as stream:
        try:
            print(f'reading from {loc}')
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

### independent variables

In [6]:
client = boto3.client('s3')

In [7]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [8]:
all_paths_config_loc = '../../config/spy_30min_v1/features.yaml'
all_paths_config = load_yaml(all_paths_config_loc)
all_paths_config.keys()

reading from ../../config/spy_30min_v1/features.yaml


dict_keys(['dependent_variable', 'feature_store'])

In [9]:
all_paths_config['feature_store']['1min'].keys()

dict_keys(['stock_bars_1min_base', 'stock_bars_1min_base_macd', 'stock_bars_1min_base_other_features', 'stock_bars_1min_base_rsi', 'stock_bars_1min_diff', 'stock_bars_1min_diff_macd', 'stock_bars_1min_diff_rsi'])

In [10]:
def get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_base', config_loc='feature_store', ):
    try:
        cols = eval(all_paths_config[config_loc][dur][file]['cols']) # stock_bars_1min_base_macd
    except:
        cols = all_paths_config[config_loc][dur][file]['cols']
    path = all_paths_config[config_loc][dur][file]['path']
    return cols, path

#### loading one min data

In [11]:
cols, path = get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_base', config_loc='feature_store', )
path, len(cols)

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet',
 87)

In [12]:
one_min_df = pq.read_pandas(path,).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
# one_min_df = one_min_df[one_min_df.symbol=='SPY']
one_min_df.head()
# df = ds.dataset(path).scanner().head(1).to_pandas()

,open,high,low,close,volume,symbol,time_delta,open_ema_2m,open_sma_2m,high_ema_2m,...,open_ema_26m,open_sma_26m,high_ema_26m,high_sma_26m,low_ema_26m,low_sma_26m,close_ema_26m,close_sma_26m,volume_ema_26m,volume_sma_26m
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,409.84,409.84,409.76,409.79,1541.0,QQQ,NaN,409.840000,NaN,409.840000,...,409.840000,NaN,409.840000,NaN,409.760000,NaN,409.790000,NaN,1541.000000,NaN
2024-01-02 04:01:00,409.81,409.81,409.75,409.75,604.0,QQQ,NaN,409.820000,409.825,409.820000,...,409.837778,NaN,409.837778,NaN,409.759259,NaN,409.787037,NaN,1471.592593,NaN
2024-01-02 04:02:00,409.71,409.71,409.68,409.68,1510.0,QQQ,NaN,409.746667,409.760,409.746667,...,409.828313,NaN,409.828313,NaN,409.753388,NaN,409.779108,NaN,1474.437586,NaN
2024-01-02 04:03:00,409.71,409.71,409.66,409.66,2679.0,QQQ,NaN,409.722222,409.710,409.722222,...,409.819549,NaN,409.819549,NaN,409.746471,NaN,409.770286,NaN,1563.664431,NaN
2024-01-02 04:04:00,409.63,409.63,409.61,409.61,343.0,QQQ,NaN,409.660741,409.670,409.660741,...,409.805508,NaN,409.805508,NaN,409.736362,NaN,409.758413,NaN,1473.244844,NaN


In [13]:
one_min_df.shape

(429593, 87)

### rsi

In [14]:
cols, path = get_paths_and_cols(all_paths_config, dur='1min', file='stock_bars_1min_base_rsi', config_loc='feature_store', )

In [15]:
cols = [x for x in cols if 'rsi' in x ]
cols

['close_7window_rsi',
 'close_9window_rsi',
 'close_11window_rsi',
 'close_14window_rsi',
 'close_17window_rsi',
 'close_21window_rsi',
 'close_ema_2m_7window_rsi',
 'close_ema_2m_9window_rsi',
 'close_ema_2m_11window_rsi',
 'close_ema_2m_14window_rsi',
 'close_ema_2m_17window_rsi',
 'close_ema_2m_21window_rsi',
 'close_sma_2m_7window_rsi',
 'close_sma_2m_9window_rsi',
 'close_sma_2m_11window_rsi',
 'close_sma_2m_14window_rsi',
 'close_sma_2m_17window_rsi',
 'close_sma_2m_21window_rsi']

In [16]:
one_min_rsi_df = pq.read_pandas(path, columns=cols).to_pandas() # pq.read_pandas('example.parquet', columns).to_pandas()
# one_min_rsi_df = one_min_rsi_df[one_min_rsi_df.symbol=='SPY']
one_min_rsi_df.head()

,close_7window_rsi,close_9window_rsi,close_11window_rsi,close_14window_rsi,close_17window_rsi,close_21window_rsi,close_ema_2m_7window_rsi,close_ema_2m_9window_rsi,close_ema_2m_11window_rsi,close_ema_2m_14window_rsi,close_ema_2m_17window_rsi,close_ema_2m_21window_rsi,close_sma_2m_7window_rsi,close_sma_2m_9window_rsi,close_sma_2m_11window_rsi,close_sma_2m_14window_rsi,close_sma_2m_17window_rsi,close_sma_2m_21window_rsi
us_eastern_timestamp,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 04:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 04:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 04:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 04:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
one_min_df.shape, one_min_rsi_df.shape

((429593, 87), (429593, 18))

In [18]:
one_min_df = pd.concat([one_min_df, one_min_rsi_df], axis=1)

In [20]:
one_min_df.shape

(429593, 105)

In [19]:
del one_min_rsi_df

In [21]:
one_min_df = one_min_df[one_min_df.symbol=='SPY']

In [22]:
one_min_df.shape

(214794, 105)

#### other features

In [23]:
try:
    cols = eval(all_paths_config['feature_store']['1min']['stock_bars_1min_base_other_features']['cols'])
except:
    cols = all_paths_config['feature_store']['1min']['stock_bars_1min_base_other_features']['cols']
path = all_paths_config['feature_store']['1min']['stock_bars_1min_base_other_features']['path']
path, len(cols)

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet',
 185)

In [24]:
cols = [x for x in cols if 'ema' not in x and 'sma' not in x and x not in ['us_eastern_timestamp', 'open', 'high', 'low', 'close', 'volume',]]
len(cols), cols

(100,
 ['open_lag_1',
  'high_lag_1',
  'low_lag_1',
  'close_lag_1',
  'open_lag_2',
  'high_lag_2',
  'low_lag_2',
  'close_lag_2',
  'open_lag_3',
  'high_lag_3',
  'low_lag_3',
  'close_lag_3',
  'open_lag_4',
  'high_lag_4',
  'low_lag_4',
  'close_lag_4',
  'open_lag_5',
  'high_lag_5',
  'low_lag_5',
  'close_lag_5',
  'open_lag_6',
  'high_lag_6',
  'low_lag_6',
  'close_lag_6',
  'open_lag_7',
  'high_lag_7',
  'low_lag_7',
  'close_lag_7',
  'open_lag_8',
  'high_lag_8',
  'low_lag_8',
  'close_lag_8',
  'open_lag_9',
  'high_lag_9',
  'low_lag_9',
  'close_lag_9',
  'open_lag_10',
  'high_lag_10',
  'low_lag_10',
  'close_lag_10',
  'open_lag_11',
  'high_lag_11',
  'low_lag_11',
  'close_lag_11',
  'open_lag_12',
  'high_lag_12',
  'low_lag_12',
  'close_lag_12',
  'open_lag_13',
  'high_lag_13',
  'low_lag_13',
  'close_lag_13',
  'open_lag_14',
  'high_lag_14',
  'low_lag_14',
  'close_lag_14',
  'open_lag_15',
  'high_lag_15',
  'low_lag_15',
  'close_lag_15',
  'market_

In [25]:
other_features = pq.read_pandas(path, columns=cols).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
other_features.head()

,open_lag_1,high_lag_1,low_lag_1,close_lag_1,open_lag_2,high_lag_2,low_lag_2,close_lag_2,open_lag_3,high_lag_3,...,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,0,0,0,0,476.36,476.0,476.36,476.0
2024-01-02 04:01:00,476.25,476.36,476.00,476.31,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,0,0,1,1,476.36,476.0,476.36,476.0
2024-01-02 04:02:00,476.34,476.34,476.29,476.29,476.25,476.36,476.00,476.31,NaN,NaN,...,False,False,0,0,2,2,476.36,476.0,476.36,476.0
2024-01-02 04:03:00,476.29,476.29,476.28,476.28,476.34,476.34,476.29,476.29,476.25,476.36,...,False,False,0,0,3,3,476.36,476.0,476.36,476.0
2024-01-02 04:04:00,476.27,476.27,476.27,476.27,476.29,476.29,476.28,476.28,476.34,476.34,...,False,False,0,0,4,4,476.36,476.0,476.36,476.0


In [26]:
one_min_df.shape[0]==other_features.shape[0]

True

In [27]:
one_min_df = pd.concat([one_min_df, other_features], axis=1, )

In [28]:
one_min_df.shape, other_features.shape

((214794, 205), (214794, 100))

In [29]:
one_min_df.head()

,open,high,low,close,volume,symbol,time_delta,open_ema_2m,open_sma_2m,high_ema_2m,...,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,SPY,NaN,476.250000,NaN,476.360000,...,False,False,0,0,0,0,476.36,476.0,476.36,476.0
2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,SPY,NaN,476.310000,476.295,476.346667,...,False,False,0,0,1,1,476.36,476.0,476.36,476.0
2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,SPY,NaN,476.296667,476.315,476.308889,...,False,False,0,0,2,2,476.36,476.0,476.36,476.0
2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,SPY,NaN,476.278889,476.280,476.282963,...,False,False,0,0,3,3,476.36,476.0,476.36,476.0
2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,SPY,NaN,476.272963,476.270,476.274321,...,False,False,0,0,4,4,476.36,476.0,476.36,476.0


### dependent_variable

In [30]:
try:
    cols = eval(all_paths_config['dependent_variable']['1min']['stock_bars_1min']['cols'])
except:
    cols = all_paths_config['dependent_variable']['1min']['stock_bars_1min']['cols']
path = all_paths_config['dependent_variable']['1min']['stock_bars_1min']['path']
path, len(cols), cols

('s3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet',
 9,
 ['open',
  'high',
  'low',
  'close',
  'close_sma_5m',
  'category',
  'future_highs',
  'future_lows',
  'slopes'])

In [31]:
dependent_var_df = pq.read_pandas(path, columns=['category']).to_pandas() # pq.read_pandas('example.parquet', columns=['two']).to_pandas()
dependent_var_df.head()

,category
us_eastern_timestamp,
2024-01-02 04:00:00,C
2024-01-02 04:01:00,C
2024-01-02 04:02:00,C
2024-01-02 04:03:00,C
2024-01-02 04:04:00,C


In [32]:
dependent_var_df.loc[:'2024-09-30'].category.value_counts()

category
C    161282
A     10399
B      8554
Name: count, dtype: int64

In [33]:
dependent_var_df.loc['2024-10-01':].category.value_counts()

category
C    31241
A     1962
B     1356
Name: count, dtype: int64

In [34]:
one_min_df.shape[0]==dependent_var_df.shape[0]

True

### final dataset

In [35]:
one_min_df = pd.concat([one_min_df, dependent_var_df['category']], axis=1).drop(columns=['symbol'])
one_min_df.head()

,open,high,low,close,volume,time_delta,open_ema_2m,open_sma_2m,high_ema_2m,high_sma_2m,...,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30,max_today,min_today,max_today_session,min_today_session,category
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,NaN,476.250000,NaN,476.360000,NaN,...,False,0,0,0,0,476.36,476.0,476.36,476.0,C
2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,NaN,476.310000,476.295,476.346667,476.350,...,False,0,0,1,1,476.36,476.0,476.36,476.0,C
2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,NaN,476.296667,476.315,476.308889,476.315,...,False,0,0,2,2,476.36,476.0,476.36,476.0,C
2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,NaN,476.278889,476.280,476.282963,476.280,...,False,0,0,3,3,476.36,476.0,476.36,476.0,C
2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,NaN,476.272963,476.270,476.274321,476.270,...,False,0,0,4,4,476.36,476.0,476.36,476.0,C


In [36]:
# one_min_df['category'] = dependent_var_df['category']
# one_min_df = one_min_df.drop(columns=['us_eastern_timestamp', 'symbol'])
# one_min_df

In [37]:
', '.join(one_min_df.columns)

'open, high, low, close, volume, time_delta, open_ema_2m, open_sma_2m, high_ema_2m, high_sma_2m, low_ema_2m, low_sma_2m, close_ema_2m, close_sma_2m, volume_ema_2m, volume_sma_2m, open_ema_3m, open_sma_3m, high_ema_3m, high_sma_3m, low_ema_3m, low_sma_3m, close_ema_3m, close_sma_3m, volume_ema_3m, volume_sma_3m, open_ema_5m, open_sma_5m, high_ema_5m, high_sma_5m, low_ema_5m, low_sma_5m, close_ema_5m, close_sma_5m, volume_ema_5m, volume_sma_5m, open_ema_8m, open_sma_8m, high_ema_8m, high_sma_8m, low_ema_8m, low_sma_8m, close_ema_8m, close_sma_8m, volume_ema_8m, volume_sma_8m, open_ema_12m, open_sma_12m, high_ema_12m, high_sma_12m, low_ema_12m, low_sma_12m, close_ema_12m, close_sma_12m, volume_ema_12m, volume_sma_12m, open_ema_10m, open_sma_10m, high_ema_10m, high_sma_10m, low_ema_10m, low_sma_10m, close_ema_10m, close_sma_10m, volume_ema_10m, volume_sma_10m, open_ema_20m, open_sma_20m, high_ema_20m, high_sma_20m, low_ema_20m, low_sma_20m, close_ema_20m, close_sma_20m, volume_ema_20m, vol

In [38]:
one_min_df.select_dtypes(include='object').columns

Index(['direction', 'category'], dtype='object')

In [39]:
one_min_df = pd.concat([one_min_df.drop(columns='direction'), pd.get_dummies(one_min_df['direction'], drop_first=True)], axis=1)

In [42]:
one_min_df.to_parquet('s3://sisyphus-general-bucket/AthenaInsights/latest_data/model/data/stock_bars_1min_rsi.parquet' )